In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Node and Tree class

In [23]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold            # Creating Constructor
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
      hist = np.bincount(y)
      ps = hist / len(y)
      return -np.sum([p * np.log(p) for p in ps if p>0])



    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [24]:
data=pd.read_csv('Threats (1).csv')
data.head()

,Unnamed: 0,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,40000,40001,0.012996,tcp,-,FIN,16,18,1540,1644,...,1,6,0,0,0,5,7,0,Normal,0
1,40001,40002,0.004119,tcp,-,FIN,12,12,1064,2260,...,1,1,0,0,0,3,8,0,Normal,0
2,40002,40003,0.000988,udp,dns,CON,2,2,146,178,...,1,2,0,0,0,7,2,0,Normal,0
3,40003,40004,3.415787,tcp,ssh,FIN,230,238,24344,29556,...,1,1,0,0,0,4,1,0,Normal,0
4,40004,40005,0.193943,tcp,-,FIN,72,74,4238,63618,...,1,5,0,0,0,6,12,0,Normal,0


# Encoding categorical data

In [25]:
data['service'] = data['service'].replace('-', 'dns')
from sklearn.preprocessing import LabelEncoder
le_proto = LabelEncoder()
le_service = LabelEncoder()
le_state = LabelEncoder()
le_attack_cat = LabelEncoder()
data['proto'] = le_proto.fit_transform(data['proto'])
data['service'] = le_service.fit_transform(data['service'])
data['state'] = le_state.fit_transform(data['state'])
data['attack_cat'] = le_attack_cat.fit_transform(data['attack_cat'])
data.head()

,Unnamed: 0,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,40000,40001,0.012996,111,1,1,16,18,1540,1644,...,1,6,0,0,0,5,7,0,6,0
1,40001,40002,0.004119,111,1,1,12,12,1064,2260,...,1,1,0,0,0,3,8,0,6,0
2,40002,40003,0.000988,117,1,0,2,2,146,178,...,1,2,0,0,0,7,2,0,6,0
3,40003,40004,3.415787,111,10,1,230,238,24344,29556,...,1,1,0,0,0,4,1,0,6,0
4,40004,40005,0.193943,111,1,1,72,74,4238,63618,...,1,5,0,0,0,6,12,0,6,0


In [26]:

cols = data.columns.tolist()

label_index = cols.index('attack_cat')
attack_index = cols.index('label')

cols[label_index], cols[attack_index] = cols[attack_index], cols[label_index]

data = data[cols]

data.head()

,Unnamed: 0,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,attack_cat
0,40000,40001,0.012996,111,1,1,16,18,1540,1644,...,1,6,0,0,0,5,7,0,0,6
1,40001,40002,0.004119,111,1,1,12,12,1064,2260,...,1,1,0,0,0,3,8,0,0,6
2,40002,40003,0.000988,117,1,0,2,2,146,178,...,1,2,0,0,0,7,2,0,0,6
3,40003,40004,3.415787,111,10,1,230,238,24344,29556,...,1,1,0,0,0,4,1,0,0,6
4,40004,40005,0.193943,111,1,1,72,74,4238,63618,...,1,5,0,0,0,6,12,0,0,6


In [27]:
X,y=data.iloc[:,:-1].values,data.iloc[:,-1].values.reshape(-1,1)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=44)

In [29]:
 #convert y_train and y_test to 1d array

y_train = y_train.flatten()
y_test = y_test.flatten()

# Training the *model*

In [31]:
clf=DecisionTree()
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)

# Perfromace metrics

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.805767637641896

In [34]:
from sklearn.metrics import f1_score
f1_score(y_test,predictions,average='weighted')

0.8060933502095488

In [35]:
from sklearn.metrics import precision_score
precision_score(y_test,predictions,average='weighted')

0.8064868701888845

In [36]:
from sklearn.metrics import recall_score
recall_score(y_test,predictions,average='weighted')

0.805767637641896